# Import relevant libraries/modules etc

In [1]:
import numpy as np
import xarray as xr
import rasterio
%matplotlib inline
from matplotlib.pyplot import *
from glob import glob
import os
import datetime
import pandas as pd
from rasterio_to_xarray import rasterio_to_xarray, xarray_to_rasterio, xarray_to_rasterio_by_band
import rasterstats

In [2]:
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler, ProgressBar
pbar = ProgressBar()
pbar.register()

# Read NetCDF files in to create dataset with all data in

In [3]:
PM25 = xr.open_mfdataset(r'C:\MAIACData\nc_monthly_daily\*PM25.nc')['data']

[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100% Completed |  0.1s
[########################################] | 100

In [4]:
PM25

<xarray.DataArray 'data' (time: 5191, y: 1162, x: 1240)>
dask.array<concate..., shape=(5191, 1162, 1240), dtype=float32, chunksize=(30, 1162, 1240)>
Coordinates:
  * x        (x) float64 -9.476e+05 -9.464e+05 -9.451e+05 -9.439e+05 ...
  * y        (y) float64 1.429e+06 1.428e+06 1.427e+06 1.426e+06 1.424e+06 ...
  * time     (time) datetime64[ns] 2000-10-01 2000-10-02 2000-10-03 ...
Attributes:
    affine: [ -9.47639631e+05   1.25654304e+03   0.00000000e+00   1.42927781e+06
   0.00000000e+00  -1.25654304e+03]
    crs: +init=epsg:27700

In [ ]:
# I think not specifying any chunks explicitly seems to work best - see above
#PM25 = xr.open_mfdataset(r'D:\MAIAC_Data\Europe\h00v01\nc_monthly\*PM25.nc', chunks={'time':5, 'x':500, 'y':500}, concat_dim='time')

# Extract data over one pixel

In [ ]:
a = PM25.attrs['affine']
a = rasterio.Affine.from_gdal(*a)
~a * (439040, 115775)

In [5]:
PM25 = PM25.isel(time=np.argsort(PM25.time))

In [6]:
After2009 = PM25.sel(time=slice('2009', '2016'))

In [7]:
monthly_data = After2009.resample('M', dim='time', how='mean', keep_attrs=True)

In [8]:
ts = monthly_data.isel(x=1103, y=1045).load()

[####                                    ] | 11% Completed |  8min  7.8s

D:\Anaconda3\lib\site-packages\dask\array\numpy_compat.py:45: RuntimeWarning: invalid value encountered in true_divide
  x = np.divide(x1, x2, out)


[########################################] | 100% Completed |  8min 19.5s


In [9]:
ts

<xarray.DataArray 'data' (time: 66)>
array([  8.01056099,   7.79006958,  13.203969  ,  14.40409565,
        13.63232994,  11.63887501,  13.61491776,  12.3930788 ,
        13.358428  ,  12.0258646 ,   7.56347275,   6.93332958,
         2.83400178,   8.09976673,   5.79116917,  12.6964016 ,
        10.23053932,   8.17398262,  15.43686581,   9.87142181,
        14.35991001,   7.34936666,   5.70906734,   3.42771339,
         6.08573294,   5.8127737 ,  25.31536102,  13.27567577,
        17.41224861,  12.60029888,  14.25756836,  12.78606415,
        14.93584347,  12.33223152,  10.96495247,   8.55024147,
         8.6985836 ,  12.28825474,  16.11894035,  13.91157818,
        14.50047112,  24.11475372,   9.81864166,  13.90888023,
        11.2921114 ,   8.53738403,   4.89646244,   6.15560246,
         7.23653412,  13.19785404,  13.38577652,  11.52484894,
        11.07980728,  16.35550499,  18.68714142,  12.74696541,
        13.93756008,  10.78917027,   7.11897087,   6.87685061,
         5.0291252

In [10]:
result = ts.to_dataframe()

In [11]:
result.dropna()

,x,y,data
time,,,
2009-01-31,438327.347127,116190.330929,8.010561
2009-02-28,438327.347127,116190.330929,7.790070
2009-03-31,438327.347127,116190.330929,13.203969
2009-04-30,438327.347127,116190.330929,14.404096
2009-05-31,438327.347127,116190.330929,13.632330
2009-06-30,438327.347127,116190.330929,11.638875
2009-07-31,438327.347127,116190.330929,13.614918
2009-08-31,438327.347127,116190.330929,12.393079
2009-09-30,438327.347127,116190.330929,13.358428


In [12]:
result.to_csv(r'D:\Annies_Dissertation\Analysis\Regression\Validation\Monthly_PM25_LSOA_Validation.csv')

# Group data
- merge multiple images per day using the maximum AOT estimates

In [ ]:
PM25 = PM25.isel(time=np.argsort(PM25.time))

In [ ]:
PM25.time

In [ ]:
m = PM25.mean(dim='time', keep_attrs=True)

In [ ]:
m = m.load()

In [ ]:
figure(figsize=(20, 8))
m.plot()

In [ ]:
from dask.dot import dot_graph

In [ ]:
##dot_graph(, format='svg')

In [ ]:
from dask.dot import to_graphviz

In [ ]:
dot_graph(m.data.dask)

In [ ]:
dot_graph(m.data.dask, filename='robin2.pdf', format='pdf')

In [ ]:
reordered_PM25 = PM25.isel(time=np.argsort(PM25.time))

In [ ]:
Daily_PM25 = reordered_PM25.resample('D', dim='time', how='max')
Daily_PM25 = Daily_PM25.dropna(dim='time', how='all')

In [ ]:
Daily_PM25.time

In [ ]:
Daily_PM25.attrs

# Analysis
## overall average of every pixel

The next four cells were added by Robin for profiling - ignore them for the moment :P

In [ ]:
om = PM25.mean(dim='time', keep_attrs=True)

In [ ]:
from dask.diagnostics import Profiler, ResourceProfiler, CacheProfiler

In [ ]:
with Profiler() as prof, ResourceProfiler(dt=0.25) as rprof:
    om.load()

In [ ]:
from dask.diagnostics import visualize
visualize([prof, rprof], show=True)

In [ ]:
overall_mean = PM25.mean(dim='time', keep_attrs=True)

In [ ]:
overall_mean.attrs

In [ ]:
with ProgressBar():
    res = overall_mean.load()

In [ ]:
xarray_to_rasterio(overall_mean, r'D:\Annies_Dissertation\Analysis\overall_mean.tif')

## Averages
### Monthly

In [ ]:
monthly_mean = PM25.groupby('time.month').mean(dim='time', keep_attrs=True)

In [ ]:
monthly_mean.coords

In [ ]:
monthly_mean.attrs

In [ ]:
figure(figsize=(20, 12))
monthly.plot(col='month', robust=True)

In [ ]:
xarray_to_rasterio_by_band(monthly_mean, r'D:\Annies_Dissertation\Analysis\monthly\monthly_mean_', dim='month')

### Seasonal

In [ ]:
seasonal_mean = PM25.groupby('time.season').mean(dim='time', keep_attrs=True)

In [ ]:
seasonal_mean.coords

In [ ]:
figure(figsize=(20, 12))
seasonal_mean.plot(col='season', robust=True)

In [ ]:
xarray_to_rasterio_by_band(seasonal_mean, r'D:\Annies_Dissertation\Analysis\seasonal\seasonal_mean_', dim='season')

### Yearly

In [ ]:
yearly_mean = PM25.groupby('time.year').mean(dim='time', keep_attrs=True)

In [ ]:
yearly_mean.coords

In [ ]:
figure(figsize=(20, 12))
yearly_mean.plot(col='year', robust=True)

In [ ]:
xarray_to_rasterio_by_band(yearly_mean, r'D:\Annies_Dissertation\Analysis\yearly\yearly_mean_', dim='year')

## Standard deviation of AP for each pixel over time- not currently what i want to do!

In [ ]:
std = PM25.groupby('time.year').std(dim = 'time', keep_attrs=True)

In [ ]:
std = PM25.std(dim = 'time', keep_attrs=True)

In [ ]:
std.coords

In [ ]:
figure(figsize=(20, 12))
std.plot(col='year', robust=True)

In [ ]:
xarray_to_rasterio(std, r'D:\Annies_Dissertation\Analysis\overall_std.tif')

# Count of values that aren't missing

In [ ]:
observations = PM25.count(dim='time', keep_attrs=True)

In [ ]:
xarray_to_rasterio(observations, r'D:\Annies_Dissertation\Analysis\observations_count.tif')

In [ ]:
count_histo = observations.plot.hist()
savefig(r'D:\Annies_Dissertation\Analysis\count_histogram.jpeg')

In [ ]:
figure(figsize=(20, 12))
observations.plot(robust=True)

# Regression
- linear regression for each pixel (Rabernat code)

In [ ]:
import xarray as xr
import numpy as np

# create an example dataset
# da = xr.DataArray(np.random.rand(10,30,40), dims=['time', 'lat', 'lon'])

# define a function to compute a linear trend of a timeseries
def linear_trend(x):
    pf = np.polyfit(x.time, x, 1)
    # we need to return a dataarray or else xarray's groupby won't be happy
    return xr.DataArray(pf[0])

In [ ]:
import xarray as xr
import numpy as np
from scipy import stats

# create an example dataset
# da = xr.DataArray(np.random.rand(10,30,40), dims=['time', 'lat', 'lon'])

# define a function to compute a linear trend of a timeseries
def linear_trend(x):
    pf = stats.linregress(x.time, x)
    # we need to return a dataarray or else xarray's groupby won't be happy
    return xr.DataArray(pf[0])

**ROBIN ADDED THIS BIT BELOW**
To subset the data to deal with the silly dask errors...

In [ ]:
# Work on a subset of the data due to dask errors!
subset = PM25.isel(x=slice(900, None), y=slice(900, None))

In [ ]:
# stack lat and lon into a single dimension called allpoints
stacked = subset.stack(allpoints=['y','x'])
# apply the function over allpoints to calculate the trend at each point
trend = stacked.groupby('allpoints').apply(linear_trend)
# unstack back to lat lon coordinates
trend_unstacked = trend.unstack('allpoints')

In [ ]:
trend_unstacked

In [ ]:
After2009 = PM25.sel(time=slice('2009', '2016'))

In [ ]:
After2009.time.min(), After2009.time.max()

# Zonal Statistics

In [ ]:
from rasterstats import zonal_stats
zonal_stats(r"D:\Annies_Dissertation\Data\LSOA\LSOA.shp", 
            r"D:\Annies_Dissertation\MAIAC_Test\subset\Projected\2014\MAIACAAOT.h00v01.20140041325.hdf_PM25.tif",
            stats="count min mean max median")

In [ ]:
from rasterstats import zonal_stats
zonal_stats(r"D:\Annies_Dissertation\Data\LSOA\LSOA.shp", 
            r"D:\Annies_Dissertation\MAIAC_Test\subset\Projected\2014\MAIACAAOT.h00v01.20140041325.hdf_PM25.tif",
            stats="count min mean max median", geojson_out=True)